In [3]:
from fastapi import FastAPI
from pydantic import BaseModel
from sklearn.pipeline import Pipeline
import uvicorn
import pandas as pd
import mlflow
import json
import joblib
from mlflow import MlflowClient
from sklearn import set_config
from pathlib import Path
#from scripts.data_clean_utils import perform_data_cleaning
from Deliveryprediction.components._3_data_cleaning import DataCleaning
from Deliveryprediction import logger


In [1]:
import os

In [2]:
os.chdir("../")

In [5]:
%pwd

'e:\\projects\\Delivery-time-prediction-for-food-devlivery-industry'

In [4]:

set_config(transform_output='pandas')

In [6]:
import dagshub
import mlflow.client
import nest_asyncio
import uvicorn


dagshub.init(repo_owner='onkar-git', 
             repo_name='Delivery-time-prediction-for-food-delivery-indust', 
             mlflow=True)

# set the mlflow tracking server
mlflow.set_tracking_uri("https://dagshub.com/onkar-git/Delivery-time-prediction-for-food-delivery-industry.mlflow")


class Data(BaseModel):  
    ID: str
    Delivery_person_ID: str
    Delivery_person_Age: str
    Delivery_person_Ratings: str
    Restaurant_latitude: float
    Restaurant_longitude: float
    Delivery_location_latitude: float
    Delivery_location_longitude: float
    Order_Date: str
    Time_Orderd: str
    Time_Order_picked: str
    Weatherconditions: str
    Road_traffic_density: str
    Vehicle_condition: int
    Type_of_order: str
    Type_of_vehicle: str
    multiple_deliveries: str
    Festival: str
    City: str

    
    
def load_model_information(file_path):
    with open(file_path) as f:
        run_info = json.load(f)
        
    return run_info


def load_transformer(transformer_path):
    transformer = joblib.load(transformer_path)
    return transformer



# columns to preprocess in data
num_cols = ["age",
            "ratings",
            "pickup_time_minutes",
            "distance"]

nominal_cat_cols = ['weather',
                    'type_of_order',
                    'type_of_vehicle',
                    "festival",
                    "city_type",
                    "is_weekend",
                    "order_time_of_day"]

ordinal_cat_cols = ["traffic","distance_type"]

#mlflow client
client = MlflowClient()

# load the model info to get the model name
model_file_path = "artifacts\model_evaluation\metrics.json"
model_name = load_model_information(model_file_path)['model_name']

# stage of the model
stage = "Staging"

# get the latest model version
# latest_model_ver = client.get_latest_versions(name=model_name,stages=[stage])
# print(f"Latest model in production is version {latest_model_ver[0].version}")

# load model path
model_path = f"models:/{model_name}/{stage}"

# load the latest model from model registry
model = mlflow.sklearn.load_model(model_path)

# load the preprocessor
preprocessor_path = "artifacts/data_trans/models/preprocessor.joblib"
preprocessor = load_transformer(preprocessor_path)

# build the model pipeline
model_pipe = Pipeline(steps=[
    ('preprocess',preprocessor),
    ("regressor",model)
])

# create the app
app = FastAPI()

# create the home endpoint
@app.get(path="/")
def home():
    return "Welcome to the Swiggy Food Delivery Time Prediction App"

# create the predict endpoint
@app.post(path="/predict")
def do_predictions(data: Data):
    pred_data = pd.DataFrame({
        'ID': data.ID,
        'Delivery_person_ID': data.Delivery_person_ID,
        'Delivery_person_Age': data.Delivery_person_Age,
        'Delivery_person_Ratings': data.Delivery_person_Ratings,
        'Restaurant_latitude': data.Restaurant_latitude,
        'Restaurant_longitude': data.Restaurant_longitude,
        'Delivery_location_latitude': data.Delivery_location_latitude,
        'Delivery_location_longitude': data.Delivery_location_longitude,
        'Order_Date': data.Order_Date,
        'Time_Orderd': data.Time_Orderd,
        'Time_Order_picked': data.Time_Order_picked,
        'Weatherconditions': data.Weatherconditions,
        'Road_traffic_density': data.Road_traffic_density,
        'Vehicle_condition': data.Vehicle_condition,
        'Type_of_order': data.Type_of_order,
        'Type_of_vehicle': data.Type_of_vehicle,
        'multiple_deliveries': data.multiple_deliveries,
        'Festival': data.Festival,
        'City': data.City
        },index=[0]
    )

    cleaned_data = DataCleaning.perform_data_cleaning(pred_data)
    # get the predictions
    predictions = model_pipe.predict(cleaned_data)[0]

    return predictions


if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app="app:app",host="0.0.0.0",port=8000)

[2025-02-20 12:06:16,316: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as onkar-git

[2025-02-20 12:06:16,331: INFO: helpers: Accessing as onkar-git]
[2025-02-20 12:06:17,395: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/onkar-git/Delivery-time-prediction-for-food-delivery-indust "HTTP/1.1 200 OK"]
[2025-02-20 12:06:18,349: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "onkar-git/Delivery-time-prediction-for-food-delivery-indust"

[2025-02-20 12:06:18,355: INFO: helpers: Initialized MLflow to track repo "onkar-git/Delivery-time-prediction-for-food-delivery-indust"]


Repository onkar-git/Delivery-time-prediction-for-food-delivery-indust initialized!

[2025-02-20 12:06:18,360: INFO: helpers: Repository onkar-git/Delivery-time-prediction-for-food-delivery-indust initialized!]


e:\projects\Delivery-time-prediction-for-food-devlivery-industry\.venv\lib\site-packages\mlflow\store\artifact\utils\models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
e:\projects\Delivery-time-prediction-for-food-devlivery-industry\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "e:\projects\Delivery-time-prediction-for-food-devlivery-industry\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3579, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_15040\1292706408.py", line 140, in <module>
    uvicorn.run(app="app:app",host="0.0.0.0",port=8000)
  File "e:\projects\Delivery-time-prediction-for-food-devlivery-industry\.venv\lib\site-packages\uvicorn\main.py", line 579, in run
    server.run()
  File "e:\projects\Delivery-time-prediction-for-food-devlivery-industry\.venv\lib\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "e:\projects\Delivery-time-prediction-for-food-devlivery-industry\.venv\lib\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "e:\projects\Delivery-time-prediction-for-food-devlivery-industry\.venv\lib\site-packages\n

In [5]:
!pip install nest_asyncio

In [7]:
import asyncio
print(asyncio.get_event_loop().is_running())


True
